In [1]:
import h5py
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import copy
import matplotlib.pyplot as plt
import os
import pandas as pd
import nibabel as nib
from nilearn import input_data, plotting,datasets
import pickle
import sys
from models import get_models
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut,KFold
from sklearn.utils import shuffle


/lustre03/project/6003287/ani686/env/lib/python3.6/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
subject=1

In [3]:
## HYPERPARAMETER CELL 

model_subject = subject
load_epoch = 3000
model_num = 1
    
LOAD_PATH = '../models/'+'sub-'+str(model_subject)+'-model-'+str(model_num)+'/'+str(load_epoch)+'.torch'
load_model = True


frame_skip = 5
sequence_len = int(90/frame_skip)
batch_size = 1

h5_filename = '../data/bw-'+'sub-'+str(subject)+'-len-'+str(sequence_len)+'.h5'#data file to train
df_filepath_stage1 = '../temp_files/'+'sub-'+str(subject)+'-stage-1-df.pkl'
runshape_file_path = '../temp_files/'+'sub-'+str(subject)+'-stage-2-runshapes-'+str(sequence_len)+'.pkl'
fmri_file_path = '../temp_files/'+'sub-'+str(subject)+'-stage-3-parcel-confounds9-nohigh.pkl'


In [4]:
if subject==1:
    exclude_runs = []
if subject==2:
    exclude_runs = []   
if subject==4:
    exclude_runs = []
if subject==6:
    exclude_runs = [0]

In [5]:
class Concatdataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)
    
dataset = datasets.fetch_atlas_basc_multiscale_2015()
atlas_filename = dataset.scale444
masker = input_data.NiftiLabelsMasker(labels_img=atlas_filename,high_pass=0.01 ,standardize=True,t_r=1.49,smoothing_fwhm=8,memory='nilearn_cache')
masker.fit()

NiftiLabelsMasker(high_pass=0.01,
                  labels_img='/home/ani686/nilearn_data/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge_basc_multiscale_sym_scale444.nii.gz',
                  memory='nilearn_cache', smoothing_fwhm=8, standardize=True,
                  t_r=1.49)

In [6]:
f = h5py.File(h5_filename,'r')
data = f['state']
label = f['action']
sess = f['session']
run = f['run']
data_all = np.array(data)
label_all = np.array(label)
sess_all = np.array(sess)
run_all = np.array(run)
main_dataset = Concatdataset(data_all,label_all,sess_all,run_all)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('frame data loaded')

model = get_models(model_num,sequence_len)
model = model.to(device)
if load_model:
    saved_state = torch.load(LOAD_PATH ,map_location=device)
    model.load_state_dict(saved_state)
    print('Loaded model',flush=True)

frame data loaded
Loaded model


In [7]:
def stage_5(model,main_dataset):
    if True:
    
        test_loader = DataLoader(dataset=main_dataset, shuffle=False, batch_size=1)
        
        conv1 = []
        conv2 = []
        conv3 = []
        conv4 = []
        lstm = []
        last = []
        label = []
        sess_all = []
        run_all = []


        if True:
            test_accuracy=0.0
            model.eval()
            
            with torch.no_grad():
                for batchid,traindata in enumerate(test_loader):
                    data,labels,sess,run = traindata
                    
                    labels = labels.reshape(-1).to(device=device)
                   
                    data = data.float().to(device=device)
                    out,[conv1_vecs,conv2_vecs,conv3_vecs,conv4_vecs,lstm_vecs,last_vecs],_ = model(data)
                    
                    
                    conv1_vecs = conv1_vecs.detach().cpu().numpy()
                    conv2_vecs = conv2_vecs.detach().cpu().numpy()
                    conv3_vecs = conv3_vecs.detach().cpu().numpy()
                    conv4_vecs = conv4_vecs.detach().cpu().numpy()
                    lstm_vecs = lstm_vecs.detach().cpu().numpy()
                    last_vecs = last_vecs.detach().cpu().numpy()
                    label_vecs = labels.detach().cpu().numpy()
                    
                    
                    conv1.append(conv1_vecs)
                    conv2.append(conv2_vecs)
                    conv3.append(conv3_vecs)
                    conv4.append(conv4_vecs)
                    lstm.append(lstm_vecs)
                    last.append(last_vecs)
                    label.append(label_vecs)
                    sess_all.append(sess)
                    run_all.append(run)
                    
                    
                    probs = torch.exp(out)                
                    top_p, top_class = probs.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            test_accuracy=test_accuracy/len(test_loader)

            print("Accuracy: %.3f"%(test_accuracy))
            all_dict = {'conv1':conv1,'conv2':conv2,'conv3':conv3,'conv4':conv4,'lstm':lstm,'last':last,'label':label,'session':sess_all,'run':run_all}
            return all_dict

In [8]:
def stage_6(dl_activations):
    if True:
        conv1 = np.array(dl_activations['conv1']) 
        conv2 = np.array(dl_activations['conv2']) 
        conv3 = np.array(dl_activations['conv3']) 
        conv4 = np.array(dl_activations['conv4']) 
        lstm = np.array(dl_activations['lstm']) 
        last = np.array(dl_activations['last'])
        label = np.array(dl_activations['label'])
        session = np.array(dl_activations['session'])
        run = np.array(dl_activations['run'])
        
        with open(runshape_file_path, 'rb') as f:
            test_shapes = pickle.load(f)
        dl_all = np.arange(conv3.shape[0])
        
        #split into runs
        dl_final = []
        start_idx = 0
        stop_idx = 0
        for i in range(len(test_shapes)):
            start_idx = stop_idx
            stop_idx = start_idx+test_shapes[i]
            dl_final.append(dl_all[start_idx:stop_idx])
        
        tsum = 0
        for i in range(len(dl_final)):
            tsum = tsum + dl_final[i].shape[0]
        #shape match check
        if not(sum(test_shapes)==dl_all.shape[0]==tsum):
            print('not pass 1')
        
        with open(fmri_file_path, 'rb') as f:
            fmri_all = pickle.load(f)
        df = pd.read_pickle(df_filepath_stage1)
        
        fmri_all2 = []
        for i in range(len(df)):
            onset_index = int(df['onset'][i]/1.49)
            fmri_all2.append(fmri_all[i][onset_index:])
        for i in range(len(fmri_all2)):
            if(fmri_all2[i].shape[0]*1.49<df['duration'][i]):
                print('not pass 2',i)
                
        fmri_all3 = []
        for i in range(len(df)):
            stop_index = int(df['duration'][i]/1.49)
            fmri_all3.append(fmri_all2[i][:stop_index]) 
        for i in range(len(fmri_all3)):
            diff = fmri_all3[i].shape[0]*1.49 - df['duration'][i]
            if not(0>diff and diff>-1.49):
                print('Not pass 3',i,diff)
          
        for i in range(len(df)):
            if(fmri_all3[i].shape[0]-dl_final[i].shape[0]<0):
                print('not pass 4',i)
          
                
        #the dl is always small above
        for i in range(len(df)):
            fmri_all3[i]= fmri_all3[i][:dl_final[i].shape[0]]
        for i in range(len(df)):
            if not(fmri_all3[i].shape[0]==dl_final[i].shape[0]):
                print('not pass 5',i)

        lag_range = [4] # each lag = 1.5 secs
        # find out common len for all lags
        # largest lag will have shortest array size
        common_length = np.zeros(len(df))
        for i in range(len(df)):
            common_length[i] = fmri_all3[i].shape[0]-max(lag_range)

        for lag in lag_range:
            #below note dl_final deals with indices
            dl_final_all = []
            fmri_final_all = []
            for i in range(len(df)):
                
                dl_final_all.append(dl_final[i][:(fmri_all3[i].shape[0]-lag)])
                fmri_final_all.append(fmri_all3[i][lag:])    

            #for same size vectors for all considered lags
            for i in range(len(df)):
                dl_final_all[i] = dl_final_all[i][int(-common_length[i]):]
                fmri_final_all[i] = fmri_final_all[i][int(-common_length[i]):]
                
            for i in range(len(df)):
                if not(dl_final_all[i].shape[0]==fmri_final_all[i].shape[0]):
                    print('not pass 6',i)
                    
            for i in range(len(df)):
                scaler = StandardScaler()
                scaler.fit(fmri_final_all[i])
                fmri_final_all[i] = scaler.transform(fmri_final_all[i])
            
            
            new_dl_final = []
            new_fmri_final = []
            for i in range(len(df)):
                if i not in exclude_runs:
                    new_dl_final.append(dl_final_all[i])
                    new_fmri_final.append(fmri_final_all[i])
                
            
            
            dl = np.concatenate(new_dl_final[:],axis=0)
            fmri = np.concatenate(new_fmri_final[:],axis=0)
            tconv1 = conv1[dl]
            tconv2 = conv2[dl]
            tconv3 = conv3[dl]
            tconv4 = conv4[dl]
            tlstm = lstm[dl]
            tlast = last[dl]
            tlabel = label[dl]
            tsession = session[dl]
            trun = run[dl]

        all_dict = {'conv1':tconv1,'conv2':tconv2,'conv3':tconv3,'conv4':tconv4,'lstm':tlstm,'last':tlast,'label':tlabel,'fmri':fmri,'session':tsession,'run':trun}
        return all_dict
        

In [9]:
dl_activations = stage_5(model,main_dataset)

Accuracy: 0.958


In [10]:
dl_fmri = stage_6(dl_activations)

In [11]:
conv1 = dl_fmri['conv1']
conv2 = dl_fmri['conv2']
conv3 = dl_fmri['conv3']
conv4 = dl_fmri['conv4']
lstm = dl_fmri['lstm']
last = dl_fmri['last']
labels = dl_fmri['label']
fmri = dl_fmri['fmri']


def transform_layer_activations2(layer,pca1=800):
    layer = layer.reshape((-1,layer.shape[-1]))
    pca = PCA(n_components=pca1)
    pca.fit(layer)
    print(pca.explained_variance_ratio_.sum())
    layer = pca.transform(layer)
    layer = layer.reshape((-1,sequence_len,layer.shape[-1]))
    layer = layer.reshape(layer.shape[0],-1)
    pca = PCA(n_components=300)
    
    pca.fit(layer)
    print(pca.explained_variance_ratio_.sum())
    layer = pca.transform(layer)
    return layer



conv1pca2 = transform_layer_activations2(conv1)
conv2pca2 = transform_layer_activations2(conv2)
conv3pca2 = transform_layer_activations2(conv3)
conv4pca2 = transform_layer_activations2(conv4,pca1=200)
lstmpca2 = transform_layer_activations2(lstm,pca1=200)
lastpca2 = transform_layer_activations2(last,pca1=64)

0.9769308020673267
0.8670185
0.9963820817489253
0.9135489
0.9793761242091712
0.8104419
0.9999999999999999
0.94022024
1.0
0.8997724
1.0
0.9958921


In [12]:
dl_fin_temp = np.concatenate((conv1pca2,conv2pca2,conv4pca2,conv3pca2,lstmpca2),axis=1)


dl_fin = dl_fin_temp
alpha = 0.5

logo = LeaveOneGroupOut()

count = 0
all_test=[]
for train_index, test_index in logo.split(dl_fin, fmri, dl_fmri['session']):
    count = count +1
    clf = Ridge(alpha=alpha,normalize=True)

    clf.fit(dl_fin[train_index], fmri[train_index])
    train_predict = clf.predict(dl_fin[train_index])
    train_true = fmri[train_index]
    r2_train = r2_score(train_true,train_predict,multioutput='raw_values').clip(min=0)

    logo2 =  LeaveOneGroupOut()
    
    for run_train_index, run_test_index in logo2.split(dl_fin[test_index],fmri[test_index], dl_fmri['run'][test_index]):
        test_predict = clf.predict(dl_fin[test_index[run_test_index]])
        test_true = fmri[test_index[run_test_index]]
        r2_test = r2_score(test_true,test_predict,multioutput='raw_values').clip(min=0)
        all_test.append(r2_test)
            
all_test = np.array(all_test[:])
mean_r2  = all_test.mean(axis=0)
            
print(np.histogram(mean_r2, bins=[0.1, 0.2, 0.3,0.4,0.5])[0])


[68 10  0  0]


In [13]:
dl_fin_temp = np.concatenate((conv1pca2,conv2pca2),axis=1)
number = 91

dl_fin = dl_fin_temp
alpha = 0.2

logo = LeaveOneGroupOut()

count = 0
all_test=[]
for train_index, test_index in logo.split(dl_fin, fmri, dl_fmri['session']):
    count = count +1
    clf = Ridge(alpha=alpha,normalize=True)

    clf.fit(dl_fin[train_index], fmri[train_index])
    train_predict = clf.predict(dl_fin[train_index])
    train_true = fmri[train_index]
    r2_train = r2_score(train_true,train_predict,multioutput='raw_values').clip(min=0)

    logo2 =  LeaveOneGroupOut()
    
    for run_train_index, run_test_index in logo2.split(dl_fin[test_index],fmri[test_index], dl_fmri['run'][test_index]):
        test_predict = clf.predict(dl_fin[test_index[run_test_index]])
        test_true = fmri[test_index[run_test_index]]
        r2_test = r2_score(test_true,test_predict,multioutput='raw_values').clip(min=0)
        all_test.append(r2_test)
            
all_test = np.array(all_test[:])
mean_r2  = all_test.mean(axis=0)
            
print(np.histogram(mean_r2, bins=[0.1, 0.2, 0.3,0.4,0.5])[0])

[40  4  0  0]


In [14]:
np.save('results/main-'+str(subject)+'-'+str(number)+'.npy',all_test)

In [15]:
dl_fin_temp = np.concatenate((conv4pca2,lstmpca2),axis=1)

number = 92
dl_fin = dl_fin_temp
alpha = 0.2

logo = LeaveOneGroupOut()

count = 0
all_test=[]
for train_index, test_index in logo.split(dl_fin, fmri, dl_fmri['session']):
    count = count +1
    clf = Ridge(alpha=alpha,normalize=True)

    clf.fit(dl_fin[train_index], fmri[train_index])
    train_predict = clf.predict(dl_fin[train_index])
    train_true = fmri[train_index]
    r2_train = r2_score(train_true,train_predict,multioutput='raw_values').clip(min=0)

    logo2 =  LeaveOneGroupOut()
    
    for run_train_index, run_test_index in logo2.split(dl_fin[test_index],fmri[test_index], dl_fmri['run'][test_index]):
        test_predict = clf.predict(dl_fin[test_index[run_test_index]])
        test_true = fmri[test_index[run_test_index]]
        r2_test = r2_score(test_true,test_predict,multioutput='raw_values').clip(min=0)
        all_test.append(r2_test)
            
all_test = np.array(all_test[:])
mean_r2  = all_test.mean(axis=0)
            
print(np.histogram(mean_r2, bins=[0.1, 0.2, 0.3,0.4,0.5])[0])

[57  3  0  0]


In [16]:
np.save('results/main-'+str(subject)+'-'+str(number)+'.npy',all_test)